In [1]:
import os
import warnings

from keras.datasets import fashion_mnist
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import keras
import matplotlib.pyplot as plt
import numpy as np

warnings.filterwarnings('ignore')
%matplotlib inline


In [2]:
# Some constants
IMG_ROWS = 28
IMG_COLS = 28
NUM_CLASSES = 10
TEST_SIZE = 0.2
RANDOM_STATE = 2018

N_TRAIN_SAMPLES = 5000
N_TEST_SAMPLES = 1000

#Model
NO_EPOCHS = 5
BATCH_SIZE = 128

EXP_ID = 'experiment_002/'


In [3]:
# Auxiliar functions
def data_preprocessing(x_data, y_data):
    x_data = x_data.astype('float32')
    out_y = to_categorical(y_data, len(np.unique(y_data)))
    out_x = np.expand_dims(x_data, axis=-1)
    
    return out_x, out_y


# Load data

In [4]:
# load data
(x_train_data, y_train_data), (x_test_data, y_test_data) = fashion_mnist.load_data()

# For this tutorial we will work on a smaller subset
train_idxs = np.random.choice(np.arange(x_train_data.shape[0]), size=N_TRAIN_SAMPLES, replace=False)
test_idxs = np.random.choice(np.arange(x_test_data.shape[0]), size=N_TEST_SAMPLES, replace=False)

x_train_data = x_train_data[train_idxs, :, :]
y_train_data = y_train_data[train_idxs]
x_test_data = x_test_data[test_idxs, :, :]
y_test_data = y_test_data[test_idxs]

# prepare the data
X, y = data_preprocessing(x_train_data, y_train_data)
X_test, y_test = data_preprocessing(x_test_data, y_test_data)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)


# Build model

In [5]:
# Model
model = Sequential()
# Add convolution 2D
model.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
# Training data generator
datagen_train = ImageDataGenerator(
    rescale=1./255,  # We also can make a rescale on the data
    horizontal_flip=True,
    rotation_range = 60,
    width_shift_range=0.2,
    height_shift_range=0.2)

# Validation data generator
datagen_val = ImageDataGenerator(
    rescale=1./255)


# Keras Callbacks

A callback is a set of functions to be applied at given stages of the training procedure. You can use callbacks to get a view on internal states and statistics of the model during training. You can pass a list of callbacks (as the keyword argument callbacks) to the .fit() method of the Sequential or Model classes.

## ModelCheckpoint

Save the model after every epoch.

In [7]:
from keras.callbacks import ModelCheckpoint

if not os.path.exists(EXP_ID):
    os.makedirs(EXP_ID)

callbacks = [
    ModelCheckpoint(filepath=os.path.join(EXP_ID, 'weights.{epoch:02d}-{val_loss:.2f}.hdf5'),
                    monitor='val_loss', 
                    verbose=1, 
                    save_best_only=False, 
                    save_weights_only=False, 
                    mode='auto')
]

# Train!
train_model = model.fit_generator(
    datagen_train.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=NO_EPOCHS,
    validation_data=datagen_val.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    validation_steps=X_val.shape[0] // BATCH_SIZE,
    callbacks=callbacks)



Epoch 1/5
30/31 [============================>.] - ETA: 0s - loss: 2.0485 - accuracy: 0.2580
Epoch 1: saving model to experiment_002\weights.01-1.51.hdf5
31/31 [==============================] - 3s 41ms/step - loss: 2.0362 - accuracy: 0.2603 - val_loss: 1.5055 - val_accuracy: 0.4453
Epoch 2/5
31/31 [==============================] - ETA: 0s - loss: 1.6883 - accuracy: 0.3729
Epoch 2: saving model to experiment_002\weights.02-1.40.hdf5
31/31 [==============================] - 1s 32ms/step - loss: 1.6883 - accuracy: 0.3729 - val_loss: 1.3976 - val_accuracy: 0.4788
Epoch 3/5
31/31 [==============================] - ETA: 0s - loss: 1.5998 - accuracy: 0.4135
Epoch 3: saving model to experiment_002\weights.03-1.28.hdf5
31/31 [==============================] - 1s 30ms/step - loss: 1.5998 - accuracy: 0.4135 - val_loss: 1.2776 - val_accuracy: 0.5424
Epoch 4/5
30/31 [============================>.] - ETA: 0s - loss: 1.4975 - accuracy: 0.4487
Epoch 4: saving model to experiment_002\weights.04-1.27

## EarlyStopping

Stop training when a monitored quantity has stopped improving.

In [8]:
from keras.callbacks import EarlyStopping

callbacks = [
    EarlyStopping(monitor='val_loss', 
                  min_delta=0, 
                  patience=2, 
                  verbose=1, 
                  mode='auto', 
                  baseline=None, 
                  restore_best_weights=False)
]

# Train!
train_model = model.fit_generator(
    datagen_train.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=NO_EPOCHS,
    validation_data=datagen_val.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    validation_steps=X_val.shape[0] // BATCH_SIZE,
    callbacks=callbacks)


Epoch 1/5
31/31 [==============================] - 1s 33ms/step - loss: 1.3674 - accuracy: 0.5044 - val_loss: 1.2216 - val_accuracy: 0.5636
Epoch 2/5
31/31 [==============================] - 1s 31ms/step - loss: 1.2927 - accuracy: 0.5227 - val_loss: 1.2652 - val_accuracy: 0.5670
Epoch 3/5
31/31 [==============================] - 1s 30ms/step - loss: 1.2646 - accuracy: 0.5359 - val_loss: 1.1055 - val_accuracy: 0.6373
Epoch 4/5
31/31 [==============================] - 1s 30ms/step - loss: 1.2243 - accuracy: 0.5628 - val_loss: 1.1500 - val_accuracy: 0.6339
Epoch 5/5
31/31 [==============================] - 1s 30ms/step - loss: 1.1882 - accuracy: 0.5648 - val_loss: 1.0776 - val_accuracy: 0.5882


## TensorBoard

TensorBoard basic visualizations.

[TensorBoard](https://www.tensorflow.org/guide/summaries_and_tensorboard) is a visualization tool provided with TensorFlow.

This callback writes a log for TensorBoard, which allows you to visualize dynamic graphs of your training and test metrics, as well as activation histograms for the different layers in your model.

If you have installed TensorFlow with pip, you should be able to launch TensorBoard from the command line:

```
tensorboard --logdir=/full_path_to_your_logs
```


In [9]:
from keras.callbacks import TensorBoard

if not os.path.exists(EXP_ID):
    os.makedirs(EXP_ID)

callbacks = [
    TensorBoard(log_dir=os.path.join(EXP_ID, 'logs'), 
                write_graph=True, 
                write_images=False)
]

# Train!
train_model = model.fit_generator(
    datagen_train.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=NO_EPOCHS,
    validation_data=datagen_val.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    validation_steps=X_val.shape[0] // BATCH_SIZE,
    callbacks=callbacks)

Epoch 1/5
31/31 [==============================] - 1s 35ms/step - loss: 1.1295 - accuracy: 0.5878 - val_loss: 1.0936 - val_accuracy: 0.6250
Epoch 2/5
31/31 [==============================] - 1s 30ms/step - loss: 1.1126 - accuracy: 0.6030 - val_loss: 1.0918 - val_accuracy: 0.6429
Epoch 3/5
31/31 [==============================] - 1s 30ms/step - loss: 1.1131 - accuracy: 0.5974 - val_loss: 1.0580 - val_accuracy: 0.6440
Epoch 4/5
31/31 [==============================] - 1s 30ms/step - loss: 1.1105 - accuracy: 0.5994 - val_loss: 1.1558 - val_accuracy: 0.6038
Epoch 5/5
31/31 [==============================] - 1s 31ms/step - loss: 1.0558 - accuracy: 0.6191 - val_loss: 1.1235 - val_accuracy: 0.6094


## CSVLogger

Callback that streams epoch results to a csv file.

In [10]:
from keras.callbacks import CSVLogger

callbacks = [
    CSVLogger(filename='train_logs.csv', 
              separator=',', 
              append=False)
]

# Train!
train_model = model.fit_generator(
    datagen_train.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=NO_EPOCHS,
    validation_data=datagen_val.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    validation_steps=X_val.shape[0] // BATCH_SIZE,
    callbacks=callbacks)

Epoch 1/5
31/31 [==============================] - 1s 35ms/step - loss: 1.0494 - accuracy: 0.6232 - val_loss: 1.0113 - val_accuracy: 0.6518
Epoch 2/5
31/31 [==============================] - 1s 33ms/step - loss: 1.0155 - accuracy: 0.6284 - val_loss: 1.0669 - val_accuracy: 0.6217
Epoch 3/5
31/31 [==============================] - 1s 30ms/step - loss: 1.0156 - accuracy: 0.6374 - val_loss: 0.9417 - val_accuracy: 0.6172
Epoch 4/5
31/31 [==============================] - 1s 31ms/step - loss: 0.9976 - accuracy: 0.6340 - val_loss: 0.9859 - val_accuracy: 0.6775
Epoch 5/5
31/31 [==============================] - 1s 31ms/step - loss: 0.9946 - accuracy: 0.6333 - val_loss: 1.0550 - val_accuracy: 0.6440


## ReduceLROnPlateau

Reduce learning rate when a metric has stopped improving.

Models often benefit from reducing the learning rate by a factor of 2-10 once learning stagnates. This callback monitors a quantity and if no improvement is seen for a 'patience' number of epochs, the learning rate is reduced.

In [11]:
from keras.callbacks import ReduceLROnPlateau

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', 
                      factor=0.1, 
                      patience=10, 
                      verbose=0, 
                      mode='auto')
]

train_model = model.fit_generator(
    datagen_train.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=NO_EPOCHS,
    validation_data=datagen_val.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    validation_steps=X_val.shape[0] // BATCH_SIZE,
    callbacks=callbacks)


Epoch 1/5
31/31 [==============================] - 1s 33ms/step - loss: 0.9830 - accuracy: 0.6475 - val_loss: 1.0166 - val_accuracy: 0.6585 - lr: 0.0010
Epoch 2/5
31/31 [==============================] - 1s 31ms/step - loss: 0.9807 - accuracy: 0.6441 - val_loss: 0.9330 - val_accuracy: 0.6641 - lr: 0.0010
Epoch 3/5
31/31 [==============================] - 1s 31ms/step - loss: 0.9448 - accuracy: 0.6601 - val_loss: 0.9253 - val_accuracy: 0.6908 - lr: 0.0010
Epoch 4/5
31/31 [==============================] - 1s 33ms/step - loss: 0.9363 - accuracy: 0.6601 - val_loss: 0.9238 - val_accuracy: 0.6719 - lr: 0.0010
Epoch 5/5
31/31 [==============================] - 1s 29ms/step - loss: 0.9174 - accuracy: 0.6663 - val_loss: 0.8623 - val_accuracy: 0.7065 - lr: 0.0010


## Combining callbacks

In [12]:
from keras.callbacks import ModelCheckpoint, TensorBoard

if not os.path.exists(EXP_ID):
    os.makedirs(EXP_ID)

callbacks = [
    ModelCheckpoint(filepath=os.path.join(EXP_ID, 'weights.{epoch:02d}-{val_loss:.2f}.hdf5'),
                    monitor='val_loss', 
                    verbose=1, 
                    save_best_only=False, 
                    save_weights_only=False, 
                    mode='auto'),
    TensorBoard(log_dir=os.path.join(EXP_ID, 'logs'), 
                write_graph=True, 
                write_images=False)
]

# Train!
train_model = model.fit_generator(
    datagen_train.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=NO_EPOCHS,
    validation_data=datagen_val.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    validation_steps=X_val.shape[0] // BATCH_SIZE,
    callbacks=callbacks)


Epoch 1/5
30/31 [============================>.] - ETA: 0s - loss: 0.9412 - accuracy: 0.6579
Epoch 1: saving model to experiment_002\weights.01-0.85.hdf5
31/31 [==============================] - 1s 34ms/step - loss: 0.9419 - accuracy: 0.6593 - val_loss: 0.8513 - val_accuracy: 0.7109
Epoch 2/5
30/31 [============================>.] - ETA: 0s - loss: 0.9145 - accuracy: 0.6653
Epoch 2: saving model to experiment_002\weights.02-0.91.hdf5
31/31 [==============================] - 1s 33ms/step - loss: 0.9111 - accuracy: 0.6663 - val_loss: 0.9070 - val_accuracy: 0.6864
Epoch 3/5
31/31 [==============================] - ETA: 0s - loss: 0.9182 - accuracy: 0.6609
Epoch 3: saving model to experiment_002\weights.03-0.88.hdf5
31/31 [==============================] - 1s 33ms/step - loss: 0.9182 - accuracy: 0.6609 - val_loss: 0.8755 - val_accuracy: 0.6864
Epoch 4/5
31/31 [==============================] - ETA: 0s - loss: 0.9176 - accuracy: 0.6617
Epoch 4: saving model to experiment_002\weights.04-0.88